<a href="https://colab.research.google.com/github/justinfmccarty/epwmorph/blob/main/morphing_routines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install meteocalc
!pip install skyfield

  Created wheel for meteocalc: filename=meteocalc-1.1.0-cp36-none-any.whl size=8196 sha256=866737f2ebb97cf3de948447d31d66c896b1b20ce5f0ac321bea4a407795d386
  Stored in directory: /root/.cache/pip/wheels/9e/34/13/83d36ecc28837e3c2a5b696542e697538e7c1025382f4ded55
Successfully built meteocalc
     |████████████████████████████████| 389kB 5.6MB/s 
     |████████████████████████████████| 40kB 5.2MB/s 
     |████████████████████████████████| 256kB 16.8MB/s 
  Created wheel for skyfield: filename=skyfield-1.34-cp36-none-any.whl size=421399 sha256=cda2b530803b091191772e6bb392094bf129d119651068baa3953e88d16b7c31
  Stored in directory: /root/.cache/pip/wheels/48/e1/ad/9e7673f3ea6061030e82535ce33a14d79fd3b54290835c60c8
  Created wheel for jplephem: filename=jplephem-2.15-cp36-none-any.whl size=46018 sha256=0230fae990c88ef95f101c7e49ae3828eee2904842a79286d19aef96c827a77a
  Stored in directory: /root/.cache/pip/wheels/01/71/9c/beaa33beb607ef67b62765324a10cd3584e30fbeef8856b7a8
Successfully built s

In [ ]:
import pandas as pd
# import metpy.calc as mpcalc
# from metpy.units import units
import numpy as np
from datetime import timedelta
import datetime as dt
import math
from meteocalc import dew_point as calcdewpt

from skyfield import api, almanac

### Configs

In [ ]:
latitude = 49.264703
longitude = -123.148943
location = 'Vancouver'
year = 2050

### Initial Data

In [ ]:
def epw_to_dataframe(weather_path):
    epw_labels = ['year', 'month', 'day', 'hour', 'minute', 'datasource', 'drybulb_C', 'dewpoint_C', 'relhum_percent',
                  'atmos_Pa', 'exthorrad_Whm2', 'extdirrad_Whm2', 'horirsky_Whm2', 'glohorrad_Whm2',
                  'dirnorrad_Whm2', 'difhorrad_Whm2', 'glohorillum_lux', 'dirnorillum_lux', 'difhorillum_lux',
                  'zenlum_lux', 'winddir_deg', 'windspd_ms', 'totskycvr_tenths', 'opaqskycvr_tenths', 'visibility_km',
                  'ceiling_hgt_m', 'presweathobs', 'presweathcodes', 'precip_wtr_mm', 'aerosol_opt_thousandths',
                  'snowdepth_cm', 'days_last_snow', 'Albedo', 'liq_precip_depth_mm', 'liq_precip_rate_Hour']

    df = pd.DataFrame(pd.read_csv(weather_path, skiprows=8, header=None, names=epw_labels).drop('datasource', axis=1))
    return df



In [ ]:
df = epw_to_dataframe('/content/CAN_BC_Vancouver.Intl.AP.718920_CWEC2016.epw')
# returns datetime objects
df.index = pd.to_datetime(df.apply(lambda row: dt.datetime(2011, 
                                                           int(row.month), 
                                                           int(row.day), 
                                                           int(row.hour)-1,
                                                           int(row.minute)), axis=1))
df.head(2)

,year,month,day,hour,minute,drybulb_C,dewpoint_C,relhum_percent,atmos_Pa,exthorrad_Whm2,extdirrad_Whm2,horirsky_Whm2,glohorrad_Whm2,dirnorrad_Whm2,difhorrad_Whm2,glohorillum_lux,dirnorillum_lux,difhorillum_lux,zenlum_lux,winddir_deg,windspd_ms,totskycvr_tenths,opaqskycvr_tenths,visibility_km,ceiling_hgt_m,presweathobs,presweathcodes,precip_wtr_mm,aerosol_opt_thousandths,snowdepth_cm,days_last_snow,Albedo,liq_precip_depth_mm,liq_precip_rate_Hour
2011-01-01 00:00:00,2011,1,1,1,0,-1.8,-4.5,80,102160,0,0,238,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2011-01-01 01:00:00,2011,1,1,2,0,-3.0,-5.3,82,102010,0,0,233,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0


In [ ]:
df[['glohorrad_Whm2', 'dirnorrad_Whm2', 'difhorrad_Whm2']].head(24)

In [ ]:
future_df = df[['month','day','hour','minute']].copy()
future_df['year'] = year

## Utilities

In [336]:
# the following are recreation of PVLIB 
# https://pvlib-python.readthedocs.io/en/stable/_modules/pvlib/solarposition.html
# or from ASU Solar Power Labs https://www.pveducation.org/pvcdrom/properties-of-sunlight/solar-time 

# def calc_simple_day_angle(dayofyear,offset=1):
#   return ((2*np.pi)/365)*(dayofyear-offset)

def calc_simple_day_angle(dayofyear,offset=1):
  return 2 * np.pi * (dayofyear-1) / 365

# def calc_bday(dayangle):
#   return dayangle - ((2*np.pi)/365) * 80

def calc_bday(dayofyear):
  return (360/365)*(dayofyear-81)

def calc_equation_of_time(bday):
  return 9.87*np.sin(np.deg2rad(2*bday))-7.53*np.cos(np.deg2rad(bday))-1.5*np.sin(np.deg2rad(bday))

def calc_local_time_meridian(utc_offset):
  # requires the utc offset for local time zone
  # https://en.wikipedia.org/wiki/List_of_UTC_time_offsets
  return 15*utc_offset

def calc_time_correction(longitude, local_time_meridian, equation_of_time):
  return 4 * (longitude - local_time_meridian) + equation_of_time

def calc_local_solar_time(local_time, time_correction):
  return local_time + (time_correction / 60)

def calc_hour_angle(solar_time):
  return (360/24) * (solar_time-12)

def calc_declination(dayofyear):
  return 23.45*np.sin(np.deg2rad((360/365)*dayofyear-81))

def calc_solar_altitude(longitude, latitude, utc_offset, dayofyear, local_time):
  dayangle = calc_simple_day_angle(dayofyear,offset=1)
  bday = calc_bday(dayangle)
  local_time_meridian = calc_local_time_meridian(utc_offset)
  equation_of_time = calc_equation_of_time(bday)
  time_correction = calc_time_correction(longitude, local_time_meridian, equation_of_time)
  solar_time = calc_local_solar_time(local_time, time_correction)
  hour_angle = calc_hour_angle(solar_time)
  declination = calc_declination(dayofyear)
  return np.rad2deg(np.arcsin(np.sin(np.deg2rad(latitude))*np.sin(np.deg2rad(declination)) \
                   +np.cos(np.deg2rad(latitude))*np.cos(np.deg2rad(declination)) \
                   *np.cos(np.deg2rad(hour_angle))))

def calc_zenith(solar_altitude):
  return 90 - solar_altitude

def calc_rel_air_mass(zenith):
  z = np.where(zenith > 90, np.nan, zenith)
  zenith_rad = np.radians(z)
  return np.cos(zenith_rad) + 0.50572 * ((6.07995 + (90 - z)) ** - 1.6364)

def calc_atmos_precip_water(dewpt):
  return math.exp(0.07 * dewpt - 0.075)

def calc_exnor(dayofyear):
  corr_factor = 1 + 0.03344 * np.cos(360 * dayofyear / 365)
  return corr_factor * 1367

def calc_atmos_brightness(fut_dfhor, rel_air_mass, extr_norrad):
  return fut_dfhor * (rel_air_mass / extr_norrad)

def calc_atmos_clearness(fut_dfhor, fut_dnor, zenith):
  z = np.where(zenith > 90, np.nan, zenith)
  zenith_rad = np.radians(z)
  return ((fut_dfhor + fut_dnor) / (fut_dfhor + 1.041 * zenith_rad**3)) / (1 + 1.041 * zenith_rad**3)

def define_clearness_bin(atmos_clearness):
  if atmos_clearness

def uas_vas_2_sfcwind(uas,vas,calm_wind_thresh=0.5,out='SPD'):
  # taken from https://xclim.readthedocs.io/en/stable/_modules/xclim/indices/_conversion.html#uas_vas_2_sfcwind
  # adapted for no xarray

  # Wind speed is the hypotenuse of "uas" and "vas"
  wind = np.hypot(uas, vas)

  # Calculate the angle
  windfromdir_math = np.degrees(np.arctan2(vas, uas))

  # Convert the angle from the mathematical standard to the meteorological standard
  windfromdir = (270 - windfromdir_math) % 360.0

  # According to the meteorological standard, calm winds must have a direction of 0°
  # while northerly winds have a direction of 360°
  # On the Beaufort scale, calm winds are defined as < 0.5 m/s
  windfromdir = np.where(windfromdir.round() == 0, 360, windfromdir)
  windfromdir = np.where(wind < calm_wind_thresh, 0, windfromdir)
  if out=='SPD':
    return wind
  elif out=='DIR':
    return windfromdir

## Dry Bulb Temperature (dbt)

In [ ]:


fut_tas = pd.Series([-5,3,5,9,13,15,17,19,24,15,11,-1])
fut_tmax = pd.Series([3,13,15,19,22,25,27,29,33,25,19,10])
fut_tmin = pd.Series([-6,1,3,6,9,10,11,10,18,13,8,-4])
hist_tas = pd.Series([-7,0,2,6,10,12,15,18,21,14,6,-3])
hist_tmax = pd.Series([2,11,10,14,20,22,25,27,26,18,13,5])
hist_tmin = pd.Series([-12,-5,-3,0,1,5,6,10,12,11,0,-10])


def means(series):
  max = series.resample('D').max().resample('M').mean().reset_index(drop=True)
  min = series.resample('D').min().resample('M').mean().reset_index(drop=True)
  mean = series.resample('D').mean().resample('M').mean().reset_index(drop=True)
  return max, min, mean

def change(fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin):
  tas = fut_tas-hist_tas
  tmax = fut_tmax-hist_tmax
  tmin = fut_tmin-hist_tmin
  return tas, tmax, tmin

def morph_dbt(df, fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin):
  months = list(range(1, 12 + 1, 1))
  dbt_max_mean, dbt_min_mean, dbt_mean = means(df['drybulb_C'])
  tas_change, tmax_change, tmin_change = change(fut_tas, hist_tas,
                                                fut_tmax, hist_tmax,
                                                fut_tmin, hist_tmin)
  tas_change = dict(zip(months, tas_change)) 
  dbt_scale = dict(zip(months, (tmax_change - tmin_change) / (dbt_max_mean - dbt_min_mean)))
  dbt_mean = dict(zip(months, dbt_mean))
  return round(df.apply(lambda x: x['drybulb_C'] + tas_change[x['month']] + dbt_scale[x['month']] * (x['drybulb_C'] - dbt_mean[x['month']]),axis=1).astype(float),1).rename("drybulb_C")

new_dbt = morph_dbt(df, fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin)

In [ ]:
future_df['drybulb_C'] = new_dbt

## Relative Humidity (relhum)

In [ ]:
fut_rh = pd.Series([35,45,55,55,78,77,95,65,70,60,80,70])
hist_rh = pd.Series([40,60,54,50,76,70,88,61,72,64,85,75])

def morph_relhum(df, fut_relhum, hist_relhum):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    relhum_change = dict(zip(months, fut_relhum - hist_relhum))
    return df.apply(lambda x: x['relhum_percent'] + relhum_change[x['month']],axis=1).rename("relhum_percent")

new_rh = np.clip(morph_relhum(df, fut_rh, hist_rh),0,100).astype(int)

In [ ]:
future_df['relhum_percent'] = new_rh

## Pressure (pr)

In [ ]:
fut_pr = pd.Series([101860,101865,101880,101880,102160,102230,101890,101950,102110,101890,101980,102380])
hist_pr = pd.Series([101890,101855,101870,101830,102100,101880,101880,101890,102160,101880,101865,102180])

def morph_pr(df, fut_pr, hist_pr):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    pr_change = dict(zip(months, fut_pr - hist_pr))
    return df.apply(lambda x: x['atmos_Pa'] + pr_change[x['month']],axis=1).rename("atmos_Pa")

new_pr = morph_pr(df, fut_pr, hist_pr).astype(int)

In [ ]:
future_df['atmos_Pa'] = new_pr

## Dew Point (dewpt)

In [ ]:
# def calc_sat_pr(pressure, dbt, rel_hum):
#     # pressure units in in Pa
#     # pressure units out in Pa
#     # temp units in C
#     mixing = mpcalc.mixing_ratio_from_relative_humidity(pressure*units.Pa,
#                                                         dbt*units.degC,
#                                                         rel_hum*units.percent)
#     return mpcalc.vapor_pressure(pressure*units.Pa, mixing)

# def calc_partial_water_pr(pressure, dbt, rel_hum):
#     # pressure units in in Pa
#     # pressure units out in kPa
#     # temp units in C
#     # rel hum in %
#     return (rel_hum * calc_sat_pr(pressure, dbt, rel_hum)) / 1000

# def morph_dewpt(epw_dewpt, epw_pressure, epw_dbt, epw_rh):
#     pw = calc_partial_water_pr(epw_pressure, epw_dbt, epw_rh)
#     if epw_dewpt>=0:
#         epw_dewpt_fut = 6.54 + 14.526 + np.log(pw) + 0.7389 * np.log(pw)**2 + 0.09486 * np.log(pw)**3 + 0.4569 * pw**0.1984
#     else:
#         epw_dewpt_fut = 6.09 + 12.608 * np.log(pw) + 0.4959 * np.log(pw)**2
#     return epw_dewpt_fut

# TODO THe above can be used to more accuratley account for colder temperatures in dew point calc

def morph_dewpt(fut_dbt,fut_rh):
  df = pd.concat([fut_dbt,fut_rh],axis=1)
  return round(df.apply(lambda x: calcdewpt(x[0], x[1]),axis=1).astype(float),1)

new_dewpt = morph_dewpt(new_dbt,new_rh)

In [ ]:
future_df['dewpoint_C'] = new_dewpt

### Working On Dewpoint V2

In [ ]:
df_new = pd.concat([new_dbt,new_rh,new_pr,new_dewpt],axis=1)

In [ ]:
df_old = df[['drybulb_C','atmos_Pa','relhum_percent','dewpoint_C']].copy()
# df_old['dewpt2'] = df_old.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
old_dbt = df_old['drybulb_C']
old_rh = df_old['relhum_percent']
df_old['dpt2'] = round(morph_dewpt(old_dbt,old_rh).astype(float),1)


In [ ]:
def calcdew_above(pw):
  return 6.54 + 14.526 + np.log(pw) + 0.7389 * np.log(pw)**2 + 0.09486 * np.log(pw)**3 + 0.4569 * pw**0.1984

def calcdew_sub(pw):
  return 6.09 + 12.608 * np.log(pw) + 0.4959 * np.log(pw)**2

In [ ]:
# calc_partial_water_pr(df_new['atmos_Pa'][2], df_new['drybulb_C'][2], df_new['relhum_percent'][2])
df_new['dewpt'] = df_new.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
df_new['vapor'] = df_new.apply(lambda x: calc_sat_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)
df_new['partial'] = df_new.apply(lambda x: calc_partial_water_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)

NameError: ignored

In [ ]:
df_new['old_dp'] = df['dewpoint_C']


In [ ]:
df_new['dewpoint_met'] = np.where(df_new['old_dp']>=0,
                                  df_new['partial'].apply(lambda x: calcdew_above(x.magnitude)),
                                  df_new['partial'].apply(lambda x: calcdew_sub(x.magnitude)))
df_new.iloc[3000:3024]

In [ ]:
df_old = df[['drybulb_C','atmos_Pa','relhum_percent','dewpoint_C']].copy()
df_old['dewpt2'] = df_old.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
# df_old['vapor'] = df_old.apply(lambda x: calc_sat_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)
# df_old['partial'] = (df_old['vapor']*(df_old['relhum_percent']/100))

# df_old['dewpoint_met'] = np.where(df_old['dewpoint_C']>=0,
#                                   df_old['partial'].apply(lambda x: calcdew_above(x.magnitude)),
#                                   df_old['partial'].apply(lambda x: calcdew_sub(x.magnitude)))


## Global Horizontal Radiation (glohor)

In [ ]:
fut_glohor = pd.Series([33,64,111,190,220,280,230,210,140,85,38,24]) #watt per m2
hist_glohor = pd.Series([36,67,109,184,208,244,243,218,146,87,39,26]) #watt per m2

def morph_glohor(df, hist_glohor, fut_glohor):
  df['single'] = 1
  months = list(range(1,12+1,1))

  month_hours = dict(zip(months, df['single'].resample('M').sum().tolist())) #hours
  month_glohor = dict(zip(months, df['glohorrad_Whm2'].resample('M').sum().tolist())) #watt-hours per m2 
  month_glohor_mean_list = []
  for key in month_hours:
    mean = (month_glohor[key])/month_hours[key]
    month_glohor_mean_list.append(mean)
  month_glohor_mean_list = dict(zip(months, month_glohor_mean_list)) #watt per m2 

  glohor_change = dict(zip(months, fut_glohor - hist_glohor))
  glohor_scale_list = []
  for key in month_glohor_mean_list:
    glohor_scale = 1 + (glohor_change[key]/month_glohor_mean_list[key])
    glohor_scale_list.append(glohor_scale)
  glohor_scale_list = dict(zip(months, glohor_scale_list))

  return df.apply(lambda x: x['glohorrad_Whm2'] * glohor_scale_list[x['month']],axis=1).rename("glohorrad_Whm2").astype(int)

new_glohor = morph_glohor(df, hist_glohor, fut_glohor)

In [ ]:
future_df['glohorrad_Whm2'] = new_glohor

## Diffuse Horizontal Radiation (dfhor)

#### Clearness, Sunrise, Sunset

In [ ]:
#define the clearness index

hours = list(range(1,8760+1,1))
days = list(range(1,365+1,1))
months = list(range(1,12+1,1))

def calc_clearness_hourly(new_glohor, exthor):
  return pd.Series(new_glohor / exthor).rename("clearness").fillna(0)

def calc_clearness_daily(new_glohor, exthor):
  daily = pd.Series(new_glohor.resample('D').sum() / exthor.resample('D').sum())
  return daily.rename("clearness").fillna(0)


clearness = calc_clearness_hourly(future_df['glohorrad_Whm2'], df['exthorrad_Whm2'])
clearness_daily = calc_clearness_daily(future_df['glohorrad_Whm2'], df['exthorrad_Whm2'])

clearness_list = dict(zip(hours, clearness.tolist()))
clearness_day_list = dict(zip(days,clearness_daily.tolist()))

In [ ]:
future_df['hourly_clearness'] = clearness
future_df['dayofyear'] = future_df.index.dayofyear
future_df['daily_clearness'] = future_df['dayofyear'].map(clearness_day_list)

In [ ]:
#define sunrise sunset
def calc_rise_set(df, latitude, longitude):
  ts = api.load.timescale()
  eph = api.load('de421.bsp')

  location = api.Topos(latitude, longitude)

  t0 = ts.utc(2011-1, 12, 31, 0)
  t1 = ts.utc(2011+1, 1, 2, 0)

  t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, location))
  times = pd.Series(t.utc_datetime()).rename('datetimes')
  times = times + timedelta(hours=-8, minutes=0)
  keys = pd.Series(y).rename('Rise_Set')
  keys = pd.Series(np.where(keys==0,'Sunset','Sunrise')).rename('Rise_Set')
  join = pd.concat([times,keys],axis=1)
  join.set_index(join['datetimes'], inplace=True)
  join['year'] = join['datetimes'].dt.year
  join['month'] = join['datetimes'].dt.month
  join['day'] = join['datetimes'].dt.day
  join['hour'] = join['datetimes'].dt.hour
  join['minute'] = 0
  join = join[join['year']==2011]
  join['Timestamp'] = join.apply(lambda row: dt.datetime(row.year, row.month, row.day, row.hour),axis=1)
  join.set_index('Timestamp', inplace=True)

  join_sub = pd.DataFrame(join['Rise_Set'])
  join_sub['dtime']=join.index
  df['dtime']=df.index
  df = df.merge(join_sub,how='left', left_on='dtime', right_on='dtime')
  df['Rise_Set'] = df['Rise_Set'].fillna('Neither')
  df.set_index(df['dtime'], inplace=True)
  return df['Rise_Set']

rise_set = calc_rise_set(df, latitude, longitude)

[#################################] 100% de421.bsp


In [ ]:
future_df['rise_set'] = rise_set
future_df['row_number'] = future_df.reset_index(drop=True).index.tolist()

In [ ]:
# this correct lead-lag in the hourly clearness index

def persistence(hourly_clearness, rise_set, row_number):
  if rise_set=='Sunrise':
    return hourly_clearness[row_number+1]
  elif rise_set=='Sunset':
    return hourly_clearness[row_number-1]
  else:
    clearness = np.where(row_number<8759,
                         hourly_clearness[row_number-1] + hourly_clearness[row_number]/2,
                         0)
    return clearness



future_df['persisted_index'] = future_df.apply(lambda x: persistence(clearness,
                                                       x['rise_set'],
                                                       x['row_number']), axis=1)

In [ ]:
def solar_geometry(df):
  df['simple_day_angle'] = df.apply(lambda x: calc_simple_day_angle(x['dayofyear']),axis=1)
  df['bday'] = df.apply(lambda x: calc_bday(x['simple_day_angle']),axis=1)
  df['equation_of_time'] = df.apply(lambda x: calc_equation_of_time(x['bday']),axis=1)
  df['local_time_meridian'] = df.apply(lambda x: calc_local_time_meridian(-8),axis=1)
  df['time_correction'] = df.apply(lambda x: calc_time_correction(longitude,
                                                                              x['local_time_meridian'],
                                                                              x['equation_of_time']),axis=1)
  df['local_solar_time'] = df.apply(lambda x: calc_local_solar_time(x['hour'],
                                                                                x['time_correction']),axis=1)
  df['hour_angle'] = df.apply(lambda x: calc_hour_angle(x['local_solar_time']),axis=1)
  df['declination'] = df.apply(lambda x: calc_declination(x['dayofyear']),axis=1)
  df['solar_alt'] = df.apply(lambda x: calc_solar_altitude(longitude,
                                                            latitude,
                                                            -8,
                                                            x['dayofyear'],
                                                            x['hour']),axis=1)
  return df

In [ ]:
future_df = solar_geometry(future_df)
future_df.head(2)

,month,day,hour,minute,exthorrad_Whm2,extdirrad_Whm2,year,drybulb_C,relhum_percent,atmos_Pa,dewpoint_C,glohorrad_Whm2,hourly_clearness,dayofyear,daily_clearness,rise_set,row_number,persisted_index,simple_day_angle,bday,equation_of_time,local_time_meridian,time_correction,local_solar_time,hour_angle,declination,solar_alt,difhorrad_Whm2,dirnorrad_Whm2,totskycvr_tenths,present_osc,present_tsc
2011-01-01 00:00:00,1,1,1,0,0,0,2050,6.0,75,102130,1.9,0,0.0,1,0.592134,Neither,0,0.0,0.0,-79.890411,-3.256235,-120,-15.852007,0.7358,-168.963002,-23.094715,-62.422671,0,0,0,0,0
2011-01-01 01:00:00,1,1,2,0,0,0,2050,6.0,77,101980,2.3,0,0.0,1,0.592134,Neither,1,0.0,0.0,-79.890411,-3.256235,-120,-15.852007,1.7358,-153.963002,-23.094715,-56.779554,0,0,0,0,0


#### Calculate *diffhor*

In [ ]:
def calc_diffhor(df):
  return df.apply(lambda x: x['glohorrad_Whm2'] * (1/(1+math.exp(-5.38 + 6.63 * x['hourly_clearness'] + 0.006 * x['local_solar_time'] - 0.007 * x['solar_alt'] + 1.75 * x['daily_clearness'] + 1.31 * x['persisted_index']))),axis=1).astype(int)

new_diffhor = calc_diffhor(future_df)

In [ ]:
future_df['difhorrad_Whm2'] = new_diffhor

### Failed Sunrise
the following was a failed and innefficient way to define sunrise/sunset

In [ ]:

ts = api.load.timescale()
eph = api.load('de421.bsp')

[#################################] 100% de421.bsp


In [ ]:
latitude = 49.264703
longitude = -123.148943

def change_coords(lat,lon):
  if lat >= 0:
    lat_dir = 'N'
  else:
    lat_dir = 'S'
  if lon >= 0:
    lon_dir = 'E'
  else:
    lon_dir = 'W'

location = api.Topos(latitude, longitude)
location

<Topos Earth latitude 49deg 15' 52.9" N longitude -123deg 08' 56.2" E>

In [ ]:
t0 = ts.utc(2021, 1, 1, 0)
t1 = ts.utc(2021, 1, 1, 23)

f = almanac.dark_twilight_day(eph, location)
t, y = almanac.find_discrete(t0, t1, f)

# print(t.utc_iso()[4], y[4]) #start of day

# print(t.utc_iso()[0], y[0]) #start of night

# t.utc_datetime()[4] + timedelta(hours=-8, minutes=0)
# t.utc_datetime()[0] + timedelta(hours=-8, minutes=0)
for ti, yi in zip(t, y):
    print(yi, '<>', ti.utc_iso(), '<>', almanac.TWILIGHTS[yi])

3 <> 2021-01-01T00:24:15Z <> Civil twilight
2 <> 2021-01-01T01:01:41Z <> Nautical twilight
1 <> 2021-01-01T01:42:20Z <> Astronomical twilight
0 <> 2021-01-01T02:21:03Z <> Night
1 <> 2021-01-01T14:11:09Z <> Astronomical twilight
2 <> 2021-01-01T14:49:50Z <> Nautical twilight
3 <> 2021-01-01T15:30:26Z <> Civil twilight
4 <> 2021-01-01T16:07:48Z <> Day


In [ ]:
for i in list(range(1,23+1,1)):
  print(i)
  t0 = ts.utc(2021, 1, 1, i)
  t1 = ts.utc(2021, 1, 1, i+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)
  
  if y.size == 0:
    print('No')
    pass
  else:
    if almanac.TWILIGHTS[y[0]]=='Day':
      print('Yes')
      rise = t.utc_datetime()[0] + timedelta(hours=-8, minutes=0)
      print('Sunrise is at {HOUR}:{MIN}'.format(HOUR=rise.hour,MIN=rise.minute))
    else:
      print('No')



In [ ]:
for i in list(range(1,23+1,1)):
  print(i)
  t0 = ts.utc(2021, 1, 1, i)
  t1 = ts.utc(2021, 1, 1, i+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)
  
  if y.size == 0:
    print('No')
    pass
  else:
    if almanac.TWILIGHTS[y[0]]=='Night':
      print('Yes')
      sunset = t.utc_datetime()[0] + timedelta(hours=-8, minutes=0)
      print('Sunset is at {HOUR}:{MIN}'.format(HOUR=sunset.hour,MIN=sunset.minute))
    else:
      print('No')

In [ ]:
# this was super innefficient 

def (year, month, day, hour):
  t0 = ts.utc(year, month, day, hour)
  t1 = ts.utc(year, month, day, hour+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)
  
  if y.size == 0:
    return 'FALSE'
  elif almanac.TWILIGHTS[y[0]]=='Day':
    return 'TRUE'
  else:
    return 'FALSE'

idx = df.index + timedelta(hours=-8, minutes=0)
solar_df = pd.DataFrame()
solar_df['datetimes'] = idx
solar_df = solar_df.set_index(solar_df['datetimes'])
solar_df['year'] = solar_df['datetimes'].dt.year
solar_df['month'] = solar_df['datetimes'].dt.month
solar_df['day'] = solar_df['datetimes'].dt.day
solar_df['hour'] = solar_df['datetimes'].dt.hour
solar_df['minute'] = solar_df['datetimes'].dt.minute
solar_df['sunrise'] = solar_df.apply(lambda x: check_sunrise(x['year'],x['month'],x['day'],x['hour']),axis=1)

In [ ]:
t.utc_datetime()[0].hour + 1

def check_sunset(year,month,day,hour):
  t0 = ts.utc(year, month, day, hour)
  t1 = ts.utc(year, month, day, hour+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)

1

## Direct Normal Radiation (dnor)

In [ ]:
def calc_dirnor(df):
  return df.apply(lambda x: (x['glohorrad_Whm2'] - x['difhorrad_Whm2'])/np.sin(np.deg2rad(x['solar_alt'])),axis=1).astype(int)

new_dirnor = calc_dirnor(future_df)

In [ ]:
future_df['dirnorrad_Whm2'] = new_dirnor

## Total Sky Cover (tsc)

In [ ]:
hist_clt = pd.Series([77.42974091,75.87249756,67.61535645,71.07409668,59.77767181,54.62646484,51.94243622,46.48913956,54.77412796,66.72571564,75.65497589,81.23233032])
fut_clt = pd.Series([81.37533951,79.21534348,66.08791351,58.96732521,56.43816757,55.67134857,38.2667293,46.55263901,49.36717606,66.99471092,77.30912781,77.71963882])

def calc_tsc(df, hist_clt, fut_clt):
  months = list(range(1,12+1,1))
  cc_change = dict(zip(months,((fut_clt - hist_clt)/10).astype(int)))
  return np.clip(df.apply(lambda x: x['totskycvr_tenths'] + cc_change[x['month']],axis=1).rename("totskycvr_tenths"),0,10).astype(int)

new_tsc = calc_tsc(df, hist_clt, fut_clt)

In [ ]:
future_df['totskycvr_tenths'] = new_tsc

In [ ]:
future_df.apply()

2011-01-01 00:00:00     0
2011-01-01 01:00:00     0
2011-01-01 02:00:00     0
2011-01-01 03:00:00     0
2011-01-01 04:00:00     0
                       ..
2011-12-31 19:00:00    10
2011-12-31 20:00:00    10
2011-12-31 21:00:00    10
2011-12-31 22:00:00    10
2011-12-31 23:00:00    10
Name: totskycvr_tenths, Length: 8760, dtype: int64

## Opaque Sky Cover (osc)

In [ ]:
future_df['present_osc'] = df['opaqskycvr_tenths']
future_df['present_tsc'] = df['totskycvr_tenths']

def calc_osc(df):
  def calc(present_tsc, totskycvr_tenths, present_osc):
    if present_tsc==0:
      return 0
    else:
      return (totskycvr_tenths * present_osc) / present_tsc
  return df.apply(lambda x: calc(x['present_tsc'],
                                   x['totskycvr_tenths'],
                                   x['present_osc']),axis=1).astype(int)
new_osc = calc_osc(future_df)

In [ ]:
future_df['opaqskycvr_tenths'] = new_osc

## Wind Speed (wspd)

In [ ]:
uas_vas = pd.DataFrame(pd.read_csv('/content/uas+vas.csv'))

hist_uas_vas = uas_vas[['uas_hist','vas_hist','date_hist']].dropna().rename(columns={'uas_hist':'uas','vas_hist':'vas'})
hist_uas_vas['Datetime'] = pd.to_datetime(hist_uas_vas['date_hist'])
hist_uas_vas = hist_uas_vas.set_index('Datetime')
hist_uas_vas = hist_uas_vas.drop(['date_hist'],axis=1)

fut_uas_vas = uas_vas[['uas_fut','vas_fut','date_fut']].dropna().rename(columns={'uas_fut':'uas','vas_fut':'vas'})
fut_uas_vas['Datetime'] = pd.to_datetime(fut_uas_vas['date_fut'])
fut_uas_vas = fut_uas_vas.set_index('Datetime')
fut_uas_vas = fut_uas_vas.drop(['date_fut'],axis=1)

hist_uas_vas_m = hist_uas_vas.groupby(hist_uas_vas.index.month).mean()
fut_uas_vas_m = fut_uas_vas.groupby(fut_uas_vas.index.month).mean()

hist_uas_vas_m['spd'] = hist_uas_vas_m.apply(lambda x: uas_vas_2_sfcwind(x['uas'],x['vas'],out='SPD'),axis=1)
hist_uas_vas_m['dir'] = hist_uas_vas_m.apply(lambda x: uas_vas_2_sfcwind(x['uas'],x['vas'],out='DIR'),axis=1)

fut_uas_vas_m['spd'] = fut_uas_vas_m.apply(lambda x: uas_vas_2_sfcwind(x['uas'],x['vas'],out='SPD'),axis=1)
fut_uas_vas_m['dir'] = fut_uas_vas_m.apply(lambda x: uas_vas_2_sfcwind(x['uas'],x['vas'],out='DIR'),axis=1)

In [367]:
fut_spd = fut_uas_vas_m['spd']
hist_spd = hist_uas_vas_m['spd']

def morph_wspd(df, fut_spd, hist_spd):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    rel_change = 100 * ((fut_spd - hist_spd) / hist_spd)
    scale_factor_wspd = dict(zip(months, 1 + (rel_change/100)))
    return df.apply(lambda x: x['windspd_ms'] * scale_factor_wspd[x['month']],axis=1).rename("windspd_ms").astype(float)

new_wspd = morph_wspd(df, fut_spd, hist_spd).astype(int)


18

In [372]:
future_df['windspd_ms'] = new_wspd

## Global Horizontal Illuminance (glohor_lux)

In [348]:
future_df['zenith'] = future_df.apply(lambda x: calc_zenith(x['solar_alt']),axis=1)
future_df['rel_air_mass'] = future_df.apply(lambda x: calc_rel_air_mass(x['zenith']),axis=1)
future_df['extnorrad_Whm2'] = future_df.apply(lambda x: calc_exnor(x['dayofyear']),axis=1)
future_df['atmos_brightness'] =  future_df.apply(lambda x: calc_atmos_brightness(x['difhorrad_Whm2'],
                                                                                 x['rel_air_mass'],
                                                                                 x['extnorrad_Whm2']),axis=1)
future_df['atmos_clearness'] =  future_df.apply(lambda x: calc_atmos_clearness(x['difhorrad_Whm2'],
                                                                               x['dirnorrad_Whm2'],
                                                                               x['zenith']),axis=1)
future_df['atmos_clearness'].head(48)

2011-01-01 00:00:00         NaN
2011-01-01 01:00:00         NaN
2011-01-01 02:00:00         NaN
2011-01-01 03:00:00         NaN
2011-01-01 04:00:00         NaN
2011-01-01 05:00:00         NaN
2011-01-01 06:00:00         NaN
2011-01-01 07:00:00         NaN
2011-01-01 08:00:00    0.000000
2011-01-01 09:00:00    1.072709
2011-01-01 10:00:00    2.559866
2011-01-01 11:00:00    3.370237
2011-01-01 12:00:00    3.920511
2011-01-01 13:00:00    4.278253
2011-01-01 14:00:00    2.339749
2011-01-01 15:00:00    7.340228
2011-01-01 16:00:00         NaN
2011-01-01 17:00:00         NaN
2011-01-01 18:00:00         NaN
2011-01-01 19:00:00         NaN
2011-01-01 20:00:00         NaN
2011-01-01 21:00:00         NaN
2011-01-01 22:00:00         NaN
2011-01-01 23:00:00         NaN
2011-01-02 00:00:00         NaN
2011-01-02 01:00:00         NaN
2011-01-02 02:00:00         NaN
2011-01-02 03:00:00         NaN
2011-01-02 04:00:00         NaN
2011-01-02 05:00:00         NaN
2011-01-02 06:00:00         NaN
2011-01-